In [8]:
%matplotlib
%load_ext autoreload
%autoreload 2

Using matplotlib backend: Qt4Agg
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import sys
sys.path.append("/mnt/Data/Workspaces/Eclipse/dataAnalysis/Sleep-current/src/")

In [10]:
import numpy as np
import pandas as pd
import scipy.signal as scsig
import matplotlib.pyplot as pl
import pywt
import colormaps as mycmps
import signale.tools as tools
import spectrum as sp
import cPickle as pkl
import seaborn as sns

In [142]:
def zScore(array):
    return(array - np.average(array))/ np.std(array)

In [4]:
x = np.random.rand(100)
p = sp.Periodogram(x,sampling=1000)
p();
pl.yscale('log')
pl.plot(p.frequencies(),p.psd)

In [156]:
lfp = pkl.load(open('/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-05_sleep3_csc4.lfp','rb'))
#lfp.ripple_recorder(sigma=7,length=7)
#print lfp.ripples.shape
#lfp.ripplePlot()

In [150]:
rippDF = pd.DataFrame(lfp.ripples,columns=['t_start', 't_end','peak value', 't_peak'])
rippDF['epoch'] = lfp.ripples.shape[0]*['NA']
for item in lfp.sws_episodes:
    before = (rippDF.t_peak - lfp.timeAxis[0]) < item[1]
    after = (rippDF.t_peak - lfp.timeAxis[0]) > item[0]
    inBetween = np.logical_and(before,after)
    Nep = rippDF['epoch'][inBetween].size
    rippDF['epoch'][inBetween] = Nep * ['sws']
for item in lfp.rem_episodes:
    before = (rippDF.t_peak - lfp.timeAxis[0]) < item[1]
    after = (rippDF.t_peak - lfp.timeAxis[0]) > item[0]
    inBetween = np.logical_and(before,after)
    Nep = rippDF['epoch'][inBetween].size
    rippDF['epoch'][inBetween] = Nep * ['rem']
#########################3
sig = []
ripidx = []
time = []
for idx in rippDF.index:
    tP = rippDF['t_peak'][idx]
    t1 = tools.findNearest(lfp.timeAxis,tP-50)
    t2 = tools.findNearest(lfp.timeAxis,tP+50)
    sig.append(lfp.signal[t1[0]:t2[0]])
    ripidx.append(200*[idx])
    time.append(np.arange(0,100,0.5))
ripidx = np.array(ripidx).flatten()
sig =np.array(sig).flatten()
time = np.array(time).flatten()
AA = pd.DataFrame(np.array([sig,time,ripidx]).T,columns=['A','T','I'])

/home/chenani/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/chenani/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [128]:
sns.tsplot(value='A',time='T',unit='I',data=AA,ci=[68, 95],color="r")
pl.plot(AA.pivot_table('A','T'),'k--')
pl.axvline(50,linewidth=0.5,color="g")

In [157]:
rippDF = pd.DataFrame(lfp.ripples,columns=['t_start', 't_end','peak value', 't_peak'])
rippDF['epoch'] = lfp.ripples.shape[0]*['NA']
for item in lfp.sws_episodes:
    before = (rippDF.t_peak - lfp.timeAxis[0]) < item[1]
    after = (rippDF.t_peak - lfp.timeAxis[0]) > item[0]
    inBetween = np.logical_and(before,after)
    Nep = rippDF['epoch'][inBetween].size
    rippDF['epoch'][inBetween] = Nep * ['sws']
for item in lfp.rem_episodes:
    before = (rippDF.t_peak - lfp.timeAxis[0]) < item[1]
    after = (rippDF.t_peak - lfp.timeAxis[0]) > item[0]
    inBetween = np.logical_and(before,after)
    Nep = rippDF['epoch'][inBetween].size
    rippDF['epoch'][inBetween] = Nep * ['rem']
#########################3
sig = []
ripidx = []
time = []
for idx in rippDF.index:
    tP = rippDF['t_peak'][idx]
    t1 = tools.findNearest(lfp.timeAxis,tP-50)
    t2 = tools.findNearest(lfp.timeAxis,tP+50)
    sig.append(lfp.signal_white[t1[0]:t2[0]])
    ripidx.append(200*[idx])
    time.append(np.arange(0,100,0.5))
ripidx = np.array(ripidx).flatten()
sig =np.array(sig).flatten()
time = np.array(time).flatten()
AA = pd.DataFrame(np.array([sig,time,ripidx]).T,columns=['A','T','I'])

/home/chenani/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/chenani/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [114]:
avgP = sp.Periodogram(avgRipp,sampling=2000)
avgP();
#pl.yscale('log')
pl.scatter(avgP.frequencies()[:25],np.log10(avgP.psd[:25]),c='r')

NameError: name 'avgRipp' is not defined

In [158]:
ff = []
for idx in np.int16(np.unique(AA.I)):
    df = AA[AA['I']==idx]
    pp = sp.Periodogram(df['A'],sampling=2000)
    pp();
    ff.append(zScore(np.log10(pp.psd[:25])))

In [147]:
for item in ff:
    pl.scatter(pp.frequencies()[:25],item,lw=0.5)

In [159]:
f,ax = pl.subplots(1,1)
sns.tsplot(np.array(ff),time=pp.frequencies()[:25],err_style="ci_bars", interpolate=False,ax=ax)

In [19]:
from math import floor

def factors(n):
 result = []
 for i in range(2,n+1): # test all integers between 2 and n
  s = 0;
  while n/i == floor(n/float(i)): # is n/i an integer?
   n = n/float(i)
   s += 1
  if s > 0:
   for k in range(s):
    result.append(i) # i is a pf s times
   if n == 1:
    return result

# test
print factors(34)

[2, 17]


In [75]:
rippDF.as_matrix()

array([[2137977.798, 2138024.298, 44868.30641350316, 2137996.298, 'sws'],
       [2138593.298, 2138617.298, 41550.08939603467, 2138605.298, 'sws'],
       [2139703.298, 2139766.798, 39015.19946678957, 2139747.798, 'sws'],
       ..., 
       [2756536.298, 2756586.298, 48815.99787421112, 2756563.798, 'NA'],
       [2758708.798, 2758841.798, 70706.93513190364, 2758743.298, 'NA'],
       [2758914.798, 2758970.298, 39778.51841056986, 2758927.298, 'NA']], dtype=object)

In [4]:
titanic = sns.load_dataset("titanic")

In [5]:
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
5,0,3,male,NaN,0,0,8.4583,Q,Third,man,True,NaN,Queenstown,no,True
6,0,1,male,54,0,0,51.8625,S,First,man,True,E,Southampton,no,True
7,0,3,male,2,3,1,21.0750,S,Third,child,False,NaN,Southampton,no,False
8,1,3,female,27,0,2,11.1333,S,Third,woman,False,NaN,Southampton,yes,False
9,1,2,female,14,1,0,30.0708,C,Second,child,False,NaN,Cherbourg,yes,False
